In [1]:
import os
import shutil
import random
from pathlib import Path

In [3]:
!wget "https://drive.usercontent.google.com/download?id=1YMlfMalleO11EQM9DCpz-rmFsAdvRMPt&export=download&authuser=0&confirm=t&uuid=c24ffb34-fa9b-47ab-bee9-d04f60d15a1d&at=AENtkXYQUhLcbm4sgo7687PqTZD8:1732802044190" -O trash_data.zip


--2024-11-28 13:55:17--  https://drive.usercontent.google.com/download?id=1YMlfMalleO11EQM9DCpz-rmFsAdvRMPt&export=download&authuser=0&confirm=t&uuid=c24ffb34-fa9b-47ab-bee9-d04f60d15a1d&at=AENtkXYQUhLcbm4sgo7687PqTZD8:1732802044190
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.215.132, 2607:f8b0:400c:c0c::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.215.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 437243053 (417M) [application/octet-stream]
Saving to: ‘trash_data.zip’

trash_data.zip      100%[===================>] 416.99M   103MB/s    in 4.0s    

2024-11-28 13:55:23 (103 MB/s) - ‘trash_data.zip’ saved [437243053/437243053]



In [4]:
!unzip -o -q trash_data.zip -d data

In [5]:
data_dir = './data/data'
output_dir = './split_data'

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

In [6]:
# Buat folder untuk split dataset
Path(output_dir).mkdir(parents=True, exist_ok=True)
for split in ['train', 'val', 'test']:
    (Path(output_dir) / split).mkdir(parents=True, exist_ok=True)

In [7]:
for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)  # misal ./data/ada_class
    if not os.path.isdir(class_dir):
        continue

    # Buat folder untuk masing-masing class di train, val, dan test
    for split in ['train', 'val', 'test']:
        split_class_dir = os.path.join(output_dir, split, class_name)
        os.makedirs(split_class_dir, exist_ok=True)

    # Dapatkan semua file gambar dalam folder class
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    random.shuffle(images)

    # Tentukan jumlah data untuk setiap split
    train_count = int(len(images) * train_ratio)
    val_count = int(len(images) * val_ratio)
    test_count = len(images) - train_count - val_count

    # Bagi dataset
    train_images = images[:train_count]
    val_images = images[train_count:train_count + val_count]
    test_images = images[train_count + val_count:]

    # Salin gambar ke folder split masing-masing
    for img in train_images:
        shutil.copy(img, os.path.join(output_dir, 'train', class_name))
    for img in val_images:
        shutil.copy(img, os.path.join(output_dir, 'val', class_name))
    for img in test_images:
        shutil.copy(img, os.path.join(output_dir, 'test', class_name))

print("Dataset berhasil dibagi!")

Dataset berhasil dibagi!


In [8]:
import tensorflow as tf

In [11]:
data_dir = './split_data'

# Parameter dataset
image_size = (150, 150)
batch_size = 32

In [12]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=f'{data_dir}/train',
    labels='inferred',
    label_mode='categorical',  # Menggunakan one-hot encoding
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=f'{data_dir}/val',
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    batch_size=batch_size,
    shuffle=False  # Tidak perlu diacak
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=f'{data_dir}/test',
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    batch_size=batch_size,
    shuffle=False  # Tidak perlu diacak
)

# Periksa struktur dataset
print("Train Dataset:", train_dataset)
print("Validation Dataset:", val_dataset)
print("Test Dataset:", test_dataset)

Found 18917 files belonging to 22 classes.
Found 2355 files belonging to 22 classes.
Found 2385 files belonging to 22 classes.
Train Dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 22), dtype=tf.float32, name=None))>
Validation Dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 22), dtype=tf.float32, name=None))>
Test Dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 22), dtype=tf.float32, name=None))>


In [13]:
train_dataset_final = train_dataset.cache().shuffle(
    1000,
    reshuffle_each_iteration=True
).prefetch(buffer_size=tf.data.AUTOTUNE)

val_dataset_final = val_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

test_dataset_final = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)


In [15]:
models = tf.keras.models
layers = tf.keras.layers

model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(150, 150, 3)),

    # Augmentasi yang lebih agresif
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2, fill_mode="nearest"),
    tf.keras.layers.RandomZoom(0.2, fill_mode="nearest"),
    tf.keras.layers.RandomTranslation(0.2, 0.2, fill_mode="nearest"),
    tf.keras.layers.RandomBrightness(0.2),
    tf.keras.layers.RandomContrast(0.2),

    tf.keras.layers.Rescaling(1./255),
    # # tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    # tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    # tf.keras.layers.Conv2D(64, (3, 3), activation="relu", kernel_regularizer=l2(0.001)),
    # tf.keras.layers.BatchNormalization()
    tf.keras.layers.MaxPooling2D(2, 2),
    # tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(256, (3, 3), activation="relu"),
    # tf.keras.layers.Conv2D(512, (3, 3), activation="relu", kernel_regularizer=l2(0.001)),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.MaxPooling2D(2, 2),
    # tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Flatten(),

    # Dense layers dengan regularisasi yang lebih kuat
    tf.keras.layers.Dense(512, activation="relu"),

    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Activation('relu'),
    # tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(22, activation='softmax')
])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ random_flip_1 (RandomFlip)           │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_rotation_1 (RandomRotation)   │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_zoom_1 (RandomZoom)           │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation_1                 │ (None, 150, 150, 3)         │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_brightness_1                  │ (None, 150, 150, 3)         │               0 │
│ (RandomBrightness)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_contrast_1 (RandomContrast)   │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling_1 (Rescaling)              │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 148, 148, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 74, 74, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 72, 72, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 36, 36, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 34, 34, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 295936)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │     151,519,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 22)                  │          11,286 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 151,901,846 (579.46 MB)

 Trainable params: 151,901,846 (579.46 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.00001
)

In [ ]:
history = model.fit(
    train_dataset_final,
    validation_data=val_dataset_final,
    epochs=10,
    verbose=2,
    callbacks=[lr_scheduler, early_stopping]
)

Epoch 1/10


674/674 - 865s - loss: 2.7164 - accuracy: 0.2036 - val_loss: 2.7982 - val_accuracy: 0.1972 - lr: 1.0000e-04 - 865s/epoch - 1s/step
Epoch 2/10
674/674 - 776s - loss: 2.4643 - accuracy: 0.2709 - val_loss: 3.1850 - val_accuracy: 0.2114 - lr: 1.1220e-04 - 776s/epoch - 1s/step
Epoch 3/10
674/674 - 758s - loss: 2.3542 - accuracy: 0.3086 - val_loss: 3.0472 - val_accuracy: 0.2253 - lr: 1.2589e-04 - 758s/epoch - 1s/step
Epoch 4/10
674/674 - 859s - loss: 2.2827 - accuracy: 0.3256 - val_loss: 3.2308 - val_accuracy: 0.2256 - lr: 1.4125e-04 - 859s/epoch - 1s/step
Epoch 5/10
674/674 - 1231s - loss: 2.2459 - accuracy: 0.3357 - val_loss: 2.4712 - val_accuracy: 0.3097 - lr: 1.5849e-04 - 1231s/epoch - 2s/step
Epoch 6/10
674/674 - 1249s - loss: 2.1932 - accuracy: 0.3525 - val_loss: 2.7077 - val_accuracy: 0.2747 - lr: 1.7783e-04 - 1249s/epoch - 2s/step
Epoch 7/10


KeyboardInterrupt: 